In [1]:
%matplotlib inline

from __future__ import absolute_import, print_function, unicode_literals, division
from sklearn.datasets import fetch_mldata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
import random as rand
import copy

In [10]:
alice = {"Interstellar":1, "Whiplash":1, "Selma":0, "Lego Movie":0, "Birdman":0}
eric = {"Interstellar":0, "Whiplash":0, "Selma":0, "Lego Movie":1, "Birdman":0}
nancy = {"Interstellar":1, "Whiplash":1, "Selma":1, "Lego Movie":0, "Birdman":1}
#sarah = {"Interstellar":0, "Whiplash":1, "Selma":0, "Lego Movie":0, "Birdman":1}
#mike = {"Interstellar":1, "Whiplash":1, "Selma":1, "Lego Movie":1, "Birdman":1}
#bob = {"Interstellar":0, "Whiplash":1, "Selma":1, "Lego Movie":0, "Birdman":1}

data = {"alice":alice, "eric":eric, "nancy":nancy}#, "sarah":sarah} "mike":mike, "bob":bob}

data = pd.DataFrame.from_dict(data)
data.head()

,alice,eric,nancy
Birdman,0,0,1
Interstellar,1,0,1
Lego Movie,0,1,0
Selma,0,0,1
Whiplash,1,0,1


In [11]:
data_array = np.array(data)
print(data_array)
print(data_array.shape)

[[0 0 1]
 [1 0 1]
 [0 1 0]
 [0 0 1]
 [1 0 1]]
(5, 3)


In [27]:
class RBM(object):
    """
    Implementation of Restricted Boltzmann Machine for binary prediction
    
    Inputs:
    Data
    Number of hidden and visible states
    Learning rate, default set to 0.1
    
    Outputs:
    Weight matrix
    Epoch number and error
    """

    def __init__(self, data, num_visible, num_hidden, learning_rate=0.1):
        
        self.data = data
        self.num_data = data.shape[0]
        self.num_hidden = num_hidden
        self.num_visible = num_visible
        self.learning_rate = learning_rate

        # weight matrix first row and first column represent the bias
        self.weights = np.random.rand(self.num_visible, self.num_hidden) 
        self.weights = np.insert(self.weights, 0, 0, axis = 0)
        self.weights = np.insert(self.weights, 0, 0, axis = 1)
    
    def _sigmoid(self,x):
        """
        Logistic Sigmoid Activation Function
        """
        return 1.0/(1.0+np.exp(-x))
    
    def _sample_h_given_v(self, v):
        """
        Predicts hidden layer given visible layer
        """
        hid = self._sigmoid(np.dot(v, self.weights))
        hidden_states = hid > np.random.rand(self.num_hidden + 1)
        return hidden_states
    
    def _sample_v_given_h(self, h):
        """
        Predicts visible layer given hidden layer
        """
        vis = self._sigmoid(np.dot(h, self.weights.T))
        visible_states = vis > np.random.rand(self.num_visible + 1)

        return visible_states
    
    def _propagate_up(self, vis):
        """
        Propagates visible layer activation to hidden layer
        """
        vis_activation = self._sigmoid(np.dot(vis, self.weights))
        sample = vis_activation > np.random.rand(self.num_data, self.num_hidden + 1)
        return sample

    def _propagate_down(self, hid):
        """
        Propagates hidden layer activation to visible layer
        """
        hidden_activation = self._sigmoid(np.dot(hid, self.weights.T))
        return hidden_activation

    
    def CDk(self, max_epochs=1000):
        """
        Trains the RBM
        """
        self.data = np.insert(self.data, 0, 1, axis=1)
        
        for epoch in xrange(0, max_epochs):
            
            # CDk positive phase
            up_data = self._propagate_up(self.data)
            up_associations = np.dot(self.data.T, up_data)
            
            # CDk negative phase
            down_vis_probs = self._propagate_down(up_data)
            #remove the bias layer
            down_vis_probs[:,0] = 1 
            down_associations = self._sigmoid(np.dot(down_vis_probs, self.weights))            
            down_associations = np.dot(down_vis_probs.T, down_associations)
            
            self.weights += self.learning_rate * \
                ((up_associations - down_associations)/self.num_data)
                
            error = np.sum((self.data - down_vis_probs) ** 2)
            #print("Epoch: ", epoch, ", Error: ", error)
            if epoch == 100000:
                return down_vis_probs

In [28]:
print("ORIGINAL ARRAY")
print(data_array)
r = RBM(data_array, 3, 3, .8)
sample = r.CDk(max_epochs=100001)
print(r.weights)

ORIGINAL ARRAY
[[0 0 1]
 [1 0 1]
 [0 1 0]
 [0 0 1]
 [1 0 1]]
[[  1.00591977   8.15772717   3.13936072   0.19460555]
 [ -3.56526119 -20.1365942    2.37252106   6.10564137]
 [ 10.40493423   0.97435142  -5.95476714  -9.73160209]
 [-12.4850535    1.16562851   7.60490638   8.90149943]]


In [29]:
sample[:,1:].T

array([[  8.64474052e-06,   9.99792056e-01,   5.08649994e-11,
          8.64474052e-06,   9.99792056e-01],
       [  4.11759526e-07,   1.55416303e-07,   9.99988570e-01,
          4.11759526e-07,   1.55416303e-07],
       [  9.99999979e-01,   9.99999932e-01,   1.21345724e-05,
          9.99999979e-01,   9.99999932e-01]])